# Many Models Forecasting (MMF)
This notebook showcases how to run MMF with local models on multiple univariate time series of daily resolution. We will use [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128#sec5) data.

### Cluster setup

We recommend using a cluster with [Databricks Runtime 16.4 LTS for ML](https://docs.databricks.com/en/release-notes/runtime/16.4lts-ml.html). The cluster can be either a single-node or multi-node CPU cluster. MMF leverages [Pandas UDF](https://docs.databricks.com/en/udf/pandas.html) under the hood and utilizes all the available resource. Make sure to set the following Spark configurations before you start your cluster: [`spark.sql.execution.arrow.enabled true`](https://spark.apache.org/docs/3.0.1/sql-pyspark-pandas-with-arrow.html#enabling-for-conversion-tofrom-pandas) and [`spark.sql.adaptive.enabled false`](https://spark.apache.org/docs/latest/sql-performance-tuning.html#adaptive-query-execution). You can do this by specifying [Spark configuration](https://docs.databricks.com/en/compute/configure.html#spark-configuration) in the advanced options on the cluster creation page.

### Install and import packages
Check out [requirements.txt](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/requirements.txt) if you're interested in the libraries we use.

In [0]:
%pip install -r ../../requirements.txt --quiet
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
logging.getLogger("py4j.clientserver").setLevel(logging.ERROR)

In [0]:
import pathlib
import pandas as pd
from datasetsforecast.m4 import M4
from mmf_sa import run_forecast

### Prepare data 
We are using [`datasetsforecast`](https://github.com/Nixtla/datasetsforecast/tree/main/) package to download M4 data. M4 dataset contains a set of time series which we use for testing MMF. Below we have written a number of custome functions to convert M4 time series to an expected format.

In [0]:
# Number of time series
n = 1000


def create_m4_daily():
    y_df, _, _ = M4.load(directory=str(pathlib.Path.home()), group="Daily")
    _ids = [f"D{i}" for i in range(1, n)]
    y_df = (
        y_df.groupby("unique_id")
        .filter(lambda x: x.unique_id.iloc[0] in _ids)
        .groupby("unique_id")
        .apply(transform_group)
        .reset_index(drop=True)
    )
    return y_df


def transform_group(df):
    unique_id = df.unique_id.iloc[0]
    if len(df) > 1020:
        df = df.iloc[-1020:]
    _start = pd.Timestamp("2020-01-01")
    _end = _start + pd.DateOffset(days=int(df.count()[0]) - 1)
    date_idx = pd.date_range(start=_start, end=_end, freq="D", name="ds")
    res_df = pd.DataFrame(data=[], index=date_idx).reset_index()
    res_df["unique_id"] = unique_id
    res_df["y"] = df.y.values
    return res_df

We are going to save this data in a delta lake table. Provide catalog and database names where you want to store the data.

In [0]:
catalog = "mmf" # Name of the catalog we use to manage our assets
db = "m4" # Name of the schema we use to manage our assets (e.g. datasets)
user = spark.sql('select current_user() as user').collect()[0]['user'] # User email address

In [0]:
# Making sure that the catalog and the schema exist
_ = spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
_ = spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{db}")

(
    spark.createDataFrame(create_m4_daily())
    .write.format("delta").mode("overwrite")
    .saveAsTable(f"{catalog}.{db}.m4_daily_train")
)

95.8MiB [00:00, 124MiB/s]
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Daily-train.csv, 95765153, bytes.
576kiB [00:00, 25.6MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Daily-test.csv, 576459, bytes.
4.34MiB [00:00, 153MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded M4-info.csv, 4335598, bytes.
100%|██████████| 3.56M/3.56M [00:00<00:00, 59.4MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded submission-Naive2.zip, 3564691, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed /root/m4/datasets/submission-Naive2.zip


Let's take a peak at the dataset:

In [0]:
display(
  spark.sql(f"select * from {catalog}.{db}.m4_daily_train where unique_id in ('D1', 'D2', 'D3', 'D4', 'D5') order by unique_id, ds")
  )

ds,unique_id,y
2020-01-01T00:00:00Z,D1,1017.1
2020-01-02T00:00:00Z,D1,1019.3
2020-01-03T00:00:00Z,D1,1017.0
2020-01-04T00:00:00Z,D1,1019.2
2020-01-05T00:00:00Z,D1,1018.7
2020-01-06T00:00:00Z,D1,1015.6
2020-01-07T00:00:00Z,D1,1018.5
2020-01-08T00:00:00Z,D1,1018.3
2020-01-09T00:00:00Z,D1,1018.4
2020-01-10T00:00:00Z,D1,1021.5


If the number of time series is larger than the number of total cores, we set `spark.sql.shuffle.partitions` to the number of cores (can also be a multiple) so that we don't under-utilize the resource.

In [0]:
if n > sc.defaultParallelism:
    sqlContext.setConf("spark.sql.shuffle.partitions", sc.defaultParallelism)

### Models
Let's configure a list of models we are going to apply to our time series for evaluation and forecasting. A comprehensive list of all supported models is available in [mmf_sa/models/README.md](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/README.md). Look for the models where `model_type: local`; these are the local models we import from [statsforecast](https://github.com/Nixtla/statsforecast) and [sktime](https://github.com/sktime/sktime). Check their documentations for the detailed description of each model. 

Some of these models perform hyperparameter optimization ([statsforecast Automatic Forecasting](https://nixtlaverse.nixtla.io/statsforecast/index.html#automatic-forecasting)) on its own for some hyperparameters. For other hyperparameters or models, you can modify the hyperparameters in [mmf_sa/models/models_conf.yaml](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/models_conf.yaml) or overwrite the default values in [mmf_sa/forecasting_conf.yaml](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/forecasting_conf.yaml). You can also introduce new hyperparameters that are supported by the base models. To do this, first add those hyperparameters under the model specification in [mmf_sa/models/models_conf.yaml](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/models_conf.yaml). Then, include these hyperparameters inside the model instantiation which happens in the model pipeline script: e.g. `StatsFcAutoArima` class in [mmf_sa/models/statsforecast/StatsFcForecastingPipeline.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/statsforecast/StatsFcForecastingPipeline.py).

In [0]:
active_models = [
    "StatsForecastBaselineWindowAverage",
    "StatsForecastBaselineSeasonalWindowAverage",
    "StatsForecastBaselineNaive",
    "StatsForecastBaselineSeasonalNaive",
    "StatsForecastAutoArima",
    "StatsForecastAutoETS",
    "StatsForecastAutoCES",
    "StatsForecastAutoTheta",
    "StatsForecastAutoTbats",
    "StatsForecastAutoMfles",
    "StatsForecastTSB",
    "StatsForecastADIDA",
    "StatsForecastIMAPA",
    "StatsForecastCrostonClassic",
    "StatsForecastCrostonOptimized",
    "StatsForecastCrostonSBA",
    "SKTimeProphet",
    ]

### Run MMF

Now, we can run the evaluation and forecasting using `run_forecast` function defined in [mmf_sa/models/__init__.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/__init__.py).

Refer to [README.md](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/README.md#parameters-description) for a comprehensive explanation of each parameter. Note that we are not providing any covariate field (i.e. `static_features`, `dynamic_future` or `dynamic_historical`) yet in this example. We will look into how we can add exogenous regressors to help our models in a different notebook: [examples/external_regressors/local_univariate_external_regressors_daily.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/examples/external_regressors/local_univariate_external_regressors_daily.py).

While the following cell is running, you can check the status of your run on Experiments. Make sure you look for the experiments with the path you provided as `experiment_path` within `run_forecast`. On the Experiments page, you see one entry per one model (i.e. StatsForecastAutoArima). The metric provided here is a simple average over all back testing trials and all time series. This is intended to give you an initial feeling of how good each model performs on your entire data mix. But we will look into how you can scrutinize the evaluation using the `evaluation_output` table in a bit. 

If you are interested in how Pandas UDF achieves parallel fitting and forecasting of multiple time series by distributing them across multiple executors, have a look at the two methods `evaluate_local_model` and `score_local_model` defined in the source code [`Forecaster.py`](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/Forecaster.py).

In [0]:
run_forecast(
    spark=spark,
    train_data=f"{catalog}.{db}.m4_daily_train",
    scoring_data=f"{catalog}.{db}.m4_daily_train",
    scoring_output=f"{catalog}.{db}.daily_scoring_output",
    evaluation_output=f"{catalog}.{db}.daily_evaluation_output",
    group_id="unique_id",
    date_col="ds",
    target="y",
    freq="D",
    prediction_length=10,
    backtest_length=30,
    stride=10,
    metric="smape",
    train_predict_ratio=1,
    data_quality_check=True,
    resample=False,
    active_models=active_models,
    experiment_path=f"/Users/{user}/mmf/m4_daily",
    use_case_name="m4_daily",
)

Run quality checks


INFO:mmf_sa.data_quality_checks:Starting data quality checks...
INFO:mmf_sa.data_quality_checks:Initial dataset: 809659 records across 999 groups
INFO:mmf_sa.data_quality_checks:Running mandatory configuration checks...
INFO:mmf_sa.data_quality_checks:Running optional data quality checks...
INFO:mmf_sa.data_quality_checks:Data quality summary:
INFO:mmf_sa.data_quality_checks:  - Initial groups: 999
INFO:mmf_sa.data_quality_checks:  - Final groups: 628
INFO:mmf_sa.data_quality_checks:  - Removed groups: 371 (37.1%)
INFO:mmf_sa.data_quality_checks:  - Removal reasons:
INFO:mmf_sa.data_quality_checks:    * missing entries detected and resampling disabled: 371 (100.0%)
INFO:mmf_sa.data_quality_checks:Data quality checks completed successfully. Final dataset: 640560 records across 628 groups


Finished quality checks
Starting evaluate_score
Starting evaluate_models
Started evaluating StatsForecastBaselineWindowAverage
  metric_name  metric_value
0       smape      0.024395
Finished evaluating StatsForecastBaselineWindowAverage
Started evaluating StatsForecastBaselineSeasonalWindowAverage
  metric_name  metric_value
0       smape      0.045497
Finished evaluating StatsForecastBaselineSeasonalWindowAverage
Started evaluating StatsForecastBaselineNaive
  metric_name  metric_value
0       smape      0.021271
Finished evaluating StatsForecastBaselineNaive
Started evaluating StatsForecastBaselineSeasonalNaive
  metric_name  metric_value
0       smape      0.027536
Finished evaluating StatsForecastBaselineSeasonalNaive
Started evaluating StatsForecastAutoArima
  metric_name  metric_value
0       smape      0.021234
Finished evaluating StatsForecastAutoArima
Started evaluating StatsForecastAutoETS
  metric_name  metric_value
0       smape      0.021265
Finished evaluating StatsForec

'c2e8ff08-21b0-404d-85e9-a60521862272'

### Evaluate
In `evaluation_output` table, the we store all evaluation results for all backtesting trials from all models. This information can be used to understand which models performed well on which time series on which periods of backtesting. This is very important for selecting the final model for forecasting or models for ensembling. Maybe, it's faster to take a look at the table:

In [0]:
display(
  spark.sql(f"""
    select * from {catalog}.{db}.daily_evaluation_output 
    where unique_id = 'D1'
    order by unique_id, model, backtest_window_start_date
    """))

unique_id,backtest_window_start_date,metric_name,metric_value,forecast,actual,model_pickle,run_id,run_date,model,use_case,model_uri
D1,2022-09-17T00:00:00Z,smape,0.02034444955034368,"List(1969.7882754067907, 1970.4296408683008, 1970.1927025732905, 1969.3860102047083, 1969.506209232326, 1969.3169484971063, 1969.325534165597, 1970.0225304080643, 1970.2193902943568, 1969.4563121784754)","List(2002.7, 2004.1, 2004.3, 2016.4, 2016.5, 2021.1, 2008.2, 2002.9, 2012.6, 2013.8)",gAWVOBYBAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBRJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,SKTimeProphet,m4_daily,
D1,2022-09-27T00:00:00Z,smape,0.021388491571280494,"List(1990.406934174949, 1990.0197686384372, 1989.2426844212353, 1988.4626034077812, 1988.5822990361514, 1988.0361475642385, 1986.4513000203144, 1984.1649617190963, 1982.8805050268738, 1981.3546068821615)","List(2003.4, 2015.6, 2009.7, 2022.1, 2031.6, 2029.7, 2039.2, 2035.0, 2051.8, 2061.8)",gAWVsBgBAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBRJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,SKTimeProphet,m4_daily,
D1,2022-10-07T00:00:00Z,smape,0.02822858116464231,"List(2011.5644581160104, 2012.0094707115402, 2011.823653671199, 2010.8461984090043, 2009.3091880818074, 2009.066662836437, 2008.5862021542914, 2008.1519927478812, 2009.285870397707, 2010.0305119992586)","List(2063.5, 2069.5, 2054.0, 2057.0, 2062.8, 2066.4, 2067.4, 2071.4, 2083.8, 2080.6)",gAWVPhsBAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBRJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,SKTimeProphet,m4_daily,
D1,2022-09-17T00:00:00Z,smape,0.007870358003785322,"List(1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836, 1994.4906144103836)","List(2002.7, 2004.1, 2004.3, 2016.4, 2016.5, 2021.1, 2008.2, 2002.9, 2012.6, 2013.8)",gAWVqmMAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAVBRElEQZSTlCmBlH2UKIwFYWxpYXOUjAVBRElEQZSMFHByZWQ= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,StatsForecastADIDA,m4_daily,
D1,2022-09-27T00:00:00Z,smape,0.01023670138315447,"List(2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434, 2011.0408965943434)","List(2003.4, 2015.6, 2009.7, 2022.1, 2031.6, 2029.7, 2039.2, 2035.0, 2051.8, 2061.8)",gAWVmmQAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAVBRElEQZSTlCmBlH2UKIwFYWxpYXOUjAVBRElEQZSMFHByZWQ= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,StatsForecastADIDA,m4_daily,
D1,2022-10-07T00:00:00Z,smape,0.0115036089131522,"List(2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069, 2043.9717698504069)","List(2063.5, 2069.5, 2054.0, 2057.0, 2062.8, 2066.4, 2067.4, 2071.4, 2083.8, 2080.6)",gAWVimUAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAVBRElEQZSTlCmBlH2UKIwFYWxpYXOUjAVBRElEQZSMFHByZWQ= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,StatsForecastADIDA,m4_daily,
D1,2022-09-17T00:00:00Z,smape,0.0024463435064908815,"List(2002.0949443117788, 2003.0898886235577, 2004.0848329353366, 2005.0797772471158, 2006.0747215

For each backtesting trial, we train the model on the training dataset only up to `backtesting_window_start_date`. We then use that fitted model to generate the forecasts for that specific horizon. We then move `backtesting_window_start_date` forward by `stride` and do the same exercise. This continues until `backtesting_window_start_date` reaches the last day of the time series given in the training dataset. See how MMF implements backtesting in `backtest` method in [mmf_sa/models/abstract_model.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/abstract_model.py).
> 
We store the **as-if** forecasts together with the actuals for each backtesting period, so you can construct any metric of your interest. We provide a few out-of-the-box metrics for you (e.g. smape), but the idea here is that you construct your own metrics reflecting your business requirements and evaluate models based on those. For example, maybe you care more about the accuracy of the near-horizon forecasts than the far-horizon ones. In such case, you can apply a decreasing wieght to compute weighted aggregated metrics.

Note that if you run other global and foundation models against the same time series with the same input parameters (except for those specifying global and foundation models), you will get the entries from those models in the same table and will be able to compare across all types models, which is the biggest benefit of having all models integrated in one solution.

We also store each model in a binary format in this table (`model_pickle`). You can unpickle the models and access their specifications or produce forecasts. 

### Forecast
In `scoring_output` table, forecasts for each time series from each model are stored. Based on the evaluation exercised performed on `evaluation_output` table, you can select the forecasts from the best performing models or a mix of models. We are again storing each model in a binary format in this table.

In [0]:
display(spark.sql(f"""
                  select * from {catalog}.{db}.daily_scoring_output 
                  where unique_id = 'D1'
                  order by unique_id, model, ds
                  """))

unique_id,ds,y,model_pickle,run_id,run_date,use_case,model,model_uri
D1,"List(2022-10-17T00:00:00Z, 2022-10-18T00:00:00Z, 2022-10-19T00:00:00Z, 2022-10-20T00:00:00Z, 2022-10-21T00:00:00Z, 2022-10-22T00:00:00Z, 2022-10-23T00:00:00Z, 2022-10-24T00:00:00Z, 2022-10-25T00:00:00Z, 2022-10-26T00:00:00Z)","List(2056.537071763098, 2057.636267337722, 2060.1030791142985, 2062.4005665441446, 2065.1556070883, 2069.238364266272, 2072.7119363067986, 2075.5807970406954, 2078.2459547399503, 2082.2748219100868)",gAWVxB0BAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBRJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,m4_daily,SKTimeProphet,
D1,"List(2022-10-17T00:00:00Z, 2022-10-18T00:00:00Z, 2022-10-19T00:00:00Z, 2022-10-20T00:00:00Z, 2022-10-21T00:00:00Z, 2022-10-22T00:00:00Z, 2022-10-23T00:00:00Z, 2022-10-24T00:00:00Z, 2022-10-25T00:00:00Z, 2022-10-26T00:00:00Z)","List(2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596, 2073.717219245596)",gAWVemYAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAVBRElEQZSTlCmBlH2UKIwFYWxpYXOUjAVBRElEQZSMFHByZWQ= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,m4_daily,StatsForecastADIDA,
D1,"List(2022-10-17T00:00:00Z, 2022-10-18T00:00:00Z, 2022-10-19T00:00:00Z, 2022-10-20T00:00:00Z, 2022-10-21T00:00:00Z, 2022-10-22T00:00:00Z, 2022-10-23T00:00:00Z, 2022-10-24T00:00:00Z, 2022-10-25T00:00:00Z, 2022-10-26T00:00:00Z)","List(2081.6436702649526, 2082.6873405299057, 2083.7310107948583, 2084.774681059811, 2085.8183513247636, 2086.8620215897163, 2087.9056918546694, 2088.949362119622, 2089.9930323845747, 2091.036702649528)",gAWVo6sAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAlBdXRvQVJJTUGUk5QpgZR9lCiMAWSUTowBRJROjAVtYXhfcJQ= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,m4_daily,StatsForecastAutoArima,
D1,"List(2022-10-17T00:00:00Z, 2022-10-18T00:00:00Z, 2022-10-19T00:00:00Z, 2022-10-20T00:00:00Z, 2022-10-21T00:00:00Z, 2022-10-22T00:00:00Z, 2022-10-23T00:00:00Z, 2022-10-24T00:00:00Z, 2022-10-25T00:00:00Z, 2022-10-26T00:00:00Z)","List(2082.653076171875, 2084.16943359375, 2085.7275390625, 2087.25439453125, 2088.808349609375, 2090.342529296875, 2091.89453125, 2093.434326171875, 2094.986083984375, 2096.5302734375)",gAWVfsgAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAdBdXRvQ0VTlJOUKYGUfZQojA1zZWFzb25fbGVuZ3RolEsHjAU= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,m4_daily,StatsForecastAutoCES,
D1,"List(2022-10-17T00:00:00Z, 2022-10-18T00:00:00Z, 2022-10-19T00:00:00Z, 2022-10-20T00:00:00Z, 2022-10-21T00:00:00Z, 2022-10-22T00:00:00Z, 2022-10-23T00:00:00Z, 2022-10-24T00:00:00Z, 2022-10-25T00:00:00Z, 2022-10-26T00:00:00Z)","List(2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996, 2080.600319875996)",gAWVMMgAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAdBdXRvRVRTlJOUKYGUfZQojA1zZWFzb25fbGVuZ3RolEsHjAU= (truncated),c2e8ff08-21b0-404d-85e9-a60521862272,2025-07-23T03:46:17.813565Z,m4_daily,StatsForecastAutoETS,
D1,"List(2022-10-17T00:00:00Z, 2022-10-18T00:00:00Z, 2022-10-19T00:00:00Z, 2022-10-20T00:00:00Z, 2022-10-21T00:00:00Z, 2022-10-22T00:00:00Z, 2022-10-23T00:00:00Z, 2022-10-24T00:00:00Z, 2022-10-25T00:00:00Z, 2022-10-26T00:00:00Z)","List(2083.6228013197137, 2084.0847291713703, 2086.145426652324, 2087.3915733189606, 2089.296891259753, 2092.29082791041, 2093.903869433124, 2094.5175537109253, 2094.9

Refer to the [notebook](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/examples/post-evaluation-analysis.ipynb) for guidance on performing fine-grained model selection after running `run_forecast`.

### Delete Tables
Let's clean up the tables.

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.daily_evaluation_output"))

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.daily_scoring_output"))